SHIFT + ENTRER pour exécuter la Cellule de code \\
TRAN-THUONG Tien-Thinh MP* Lycée Hoche \\
5 Janvier 2022

# Importation

In [1]:
import random, hashlib, math
import sympy.ntheory as nt

# RSA

## Fonctions Utiles

In [2]:
def pgcd(a, b):
    if a==0:
        return b
    else:
        return pgcd(b%a, a)

In [3]:
def exp(a, n, mod):
    a_ = (a*a)%mod
    if n==0:
        return 1
    elif n%2 == 1:
        return (a* exp(a_, n//2, mod)) % mod
    else:
        return exp(a_, n//2, mod) % mod

In [4]:
def c_bezout(a, b):
    if a==0:
        return 0, 1
    else:
        x, y = c_bezout(b%a, a)
        return y-(x*(b//a)), x

## Calculs des clés

In [5]:
def calcul_cles(p, q):
    n = p*q
    w = (p-1)*(q-1)

    e = random.randint(2, w)
    while pgcd(e, w)!=1:
        e = random.randint(2, w)
    
    d, _ = c_bezout(e, w)
    d = d % w
    return (
        (e, n), # clé publique
        (d, n)  # clé privée
    )

In [6]:
def chiffrer(m, e, n): # dechiffrer revient à chiffrer mais avec la clé (d, n)
    return exp(m, e, n) # (m**e) % n

## Test sur des chiffres

In [7]:
(e, n), (d, _) = calcul_cles(23, 17)
e, d, n

(311, 103, 391)

In [8]:
m = 123
m_chiffre = chiffrer(m, e, n)
m_chiffre

98

In [9]:
chiffrer(m_chiffre, d, n)

123

## Test avec du TEXT

In [10]:
def list_vers_str(l):
    return "".join(l)

In [11]:
def ASCII(m):
    return [ord(x) for x in m]

def ASCII_rev(L):
    return list_vers_str([chr(x) for x in L])

In [12]:
def chiffrer_list(L, e, n):
    return [chiffrer(x, e, n) for x in L]

In [13]:
def chiffrer_message(m, e, n):
    L = ASCII(m)
    L_chiffre = chiffrer_list(L, e, n)
    m_chiffre = ASCII_rev(L_chiffre)
    return m_chiffre

In [14]:
message = "Hello World !"
m_chiffre = chiffrer_message(message, e, n)
m_chiffre

'ê\x1000½ö\x80½\xa00\x90öŤ'

In [15]:
chiffrer_message(m_chiffre, d, n) # déchiffrage

'Hello World !'

## Signature

In [16]:
p = 8949017760404478433660502120640172166331527020395781144347740356234111327237
q = 38290378838305098522902362305811701143775546488597621962754450314020740876921
nt.isprime(p), nt.isprime(q)

(True, True)

In [17]:
(e_BOB, n_BOB), (d_BOB, _) = calcul_cles(p, q)
e_BOB, d_BOB, n_BOB

(106632056836537926908100413610749097480469316880209747392147678552117385842389514750855438977593872545937999847891295591896498280638450357577085743253999,
 206755523661188422488150739072775261427078021970587901857668650117146990657409381095012728546864231214422716383017887858055916762131512919199129128407279,
 342661280276608127436787890069933716420671175404199881114217772065667627611747997598257594763995233862480669310130329426739666144892027800518034571997277)

In [18]:
def H256(x):
    binaire = str(x).encode("ASCII")
    Hash_hex = hashlib.sha256(binaire).hexdigest()
    return int(Hash_hex, 16)

m_chiffre = "Hello World !"
hash = H256(m_chiffre)
hash

3595077808819802685222014919469970274423380117249128644280510508306987122247

In [19]:
hash_chiffre = chiffrer(hash, d_BOB, n_BOB)
hash_chiffre

315351897785736084717896923848420819269939521901447125541541550074314182274741789752979662606177976925989482052690597563353404835344706841856706646124218

In [20]:
hash_dechiffre = chiffrer(hash_chiffre, e_BOB, n_BOB)
hash_dechiffre

3595077808819802685222014919469970274423380117249128644280510508306987122247

In [21]:
hash == hash_dechiffre

True

# Tests de primalité

## Méthode naïve (déterministe)

In [22]:
def est_premier(n):
    for x in range(2, int(math.sqrt(n)+1)):
        if n%x == 0:
            return False
    return True
{i: est_premier(i) for i in range(2, 15)}

{2: True,
 3: True,
 4: False,
 5: True,
 6: False,
 7: True,
 8: False,
 9: False,
 10: False,
 11: True,
 12: False,
 13: True,
 14: False}

## Crible d'Erathostene (déterministe)

In [26]:
def erathostene(n):
    L = [i for i in range(n, 1, -1)] # [n, n-1, ..., 3, 2]
    resultat = []
    while L!=[]:
        p = L.pop()
        resultat.append(p)
        L = [x for x in L if x%p != 0]
    return resultat
erathostene(50)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]

In [46]:
L = erathostene(500_000) # 1 minute 56 secondes
len(L) # 41538

41538

## Test de Primalité de Fermat (probabiliste)
Pour la base 2, il a seulement 2183 faux positifs quand on teste jusqu'à, 25 milliards. \\
Jusqu'à 500 000 il n'y a eu que 13 faux positifs sur les 41 551 nombres premiers trouvés, soit un taux de bonnes réponses de 99.969%.

In [47]:
def test_fermat_PGP(p):
    if p in [2, 3, 5, 7]:
        return True
    for a in [2, 3, 5, 7]:
        if exp(a, (p-1), p) != 1:
            return False
    return True

def gen_fermat_PGP(n):
    return [p for p in range(2, n) if test_fermat_PGP(p)]

L_ = gen_fermat_PGP(500_000) # 5 secondes
len(L_)

41551

In [58]:
print(f"Taux de bonnes réponses : {round(100*(1-(len(L_)-len(L))/len(L_)), 3)}%")

Taux de bonnes réponses : 99.969%


In [68]:
def test_fermat(p, k):
    if not test_fermat_PGP(p):
        return False
    for _ in range(k-4):
        a = random.randint(1, p-1)
        if exp(a, (p-1), p) != 1:
            return False
    return True

def gen_fermat(n, k):
    return [p for p in range(2, n) if test_fermat(p, k)]

L_ = gen_fermat(500_000, 54) # 25 secondes
len(L_)

41538

In [71]:
def gen_list_fermat(maxi, n, k):
    L = []
    while len(L) < n:
        p = random.randint(2, maxi)
        if test_fermat(p, 100):
            L.append(p)
    return L
L = gen_list_fermat(random.getrandbits(256), 2, 54)
L

[5521942464509987657915714689219924770028330532684059662498872951119193857613,
 1417027458486311454566792739389999839661991975370780538132129310429637643519]

In [72]:
[nt.isprime(x) for x in L]

[True, True]

## Test de primalité de Miller-Rabin (probabiliste)

In [77]:
def calcul_d_s(n):
    if n%2 == 1:
        return n, 0 # = d, s
    else:
        d, s = calcul_d_s(n//2)
        return d, s+1

In [83]:
def test_MR(d, s, n):
	a = random.randint(1, n - 1)
	x = exp(a, d, n)
	if (x == 1 or x == n - 1):
		return True
	for i in range(s-1):
		x = (x * x) % n
		if (x == 1):
			return False
		if (x == n - 1):
			return True
	return False

In [84]:
def test_MR_kfois( n, k):
	d, s = calcul_d_s(n-1)
	for _ in range(k):
		if not test_MR(d, s, n):
			return False
	return True

In [85]:
def gen_list_MR(maxi, n, k):
    L = []
    while len(L) < n:
        p = random.randint(2, maxi)
        if test_MR_kfois(p, k):
            L.append(p)
    return L
L = gen_list_MR(random.getrandbits(256), 10, 10)
L

[24940258735761228717503052150025314764176397447368572470357151760191775297031,
 22301253256512548890217680016435148771451730798361464134913343590242555008989,
 15737637214254841826363754682072251127306267458771856826861304580714090850603,
 10635459215469047815460593883132431656879156617527791508102668689017305496559,
 14508372617566005877662906284414037216418964589696459524597418491120888913967,
 8285894270370265487183679292853351539774979167327608178560686063556820303621,
 23977607725535021143365080042824315394829771740490719721267768394913703026111,
 552086242303829907204709363656872888392122906841463897881612395167869636529,
 20476586539208163361875836930125903978207445671547335866725766519814901368513,
 28905770571622207118930054426618318345202515192248873969720545872620148816407]

In [86]:
[nt.isprime(x) for x in L]

[True, True, True, True, True, True, True, True, True, True]

In [93]:
def gen_MR(n, k):
    return [p for p in range(2, n) if test_MR_kfois(p, k)]

L_ = gen_MR(500_000, 2) # 6 secondes
len(L_)

41545

In [95]:
L_ = gen_MR(10_000_000, 2) # 2 minutes 22 secondes
len(L_) # 664587

664587

In [98]:
L_ = gen_fermat(10_000_000, 6) # 2 minutes 20 secondes
len(L_) # 664630

664630

# Algorithme de factorisation

In [ ]:
def facto(n):
    if n == 1:
        return [1]
    elif test_MR_kfois(n, 10):
        return [n]
    for x in range(2, int(n**0.5)+1):
        if n % x == 0:
            return [x] + facto(n//x)
    return [n]
n = random.getrandbits(32)
print(f"La décomposition de {n} est {facto(n)}")

La décomposition de 897350508 est [2, 2, 3, 3, 3, 17, 47, 10399]
